<a href="https://colab.research.google.com/github/atwine/Xente-Project/blob/master/Keras_pandas_for_deeplearning_for_structured_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Keras-Pandas

There is this issue, I have had trying to use deep learning to work on structured data.

I have made some mistakes before thinking this is straight forward as just inserting in the data and let the neural nets do the rest.

However I have discovered that I have to transform the data especially the categorical variables through entity embedding in order for the neural nets to receive the data well.

This module Keras Pandas assist in doing this and below is an example of how to accomplish this using the dataset of Titanic.

For more information and documentation on how this library works one can look at this [repository of keras-pandas](https://github.com/bjherger/keras-pandas)

In [1]:
#let's install the library
!pip install -U keras-pandas

     |████████████████████████████████| 40kB 6.9MB/s 
     |████████████████████████████████| 5.3MB 18.9MB/s 
     |████████████████████████████████| 13.9MB 31.3MB/s 
     |████████████████████████████████| 8.9MB 35.6MB/s 
     |████████████████████████████████| 317kB 30.0MB/s 
     |████████████████████████████████| 63.0MB 794kB/s 
     |████████████████████████████████| 61kB 14.1MB/s 
     |████████████████████████████████| 573kB 33.8MB/s 
     |████████████████████████████████| 3.0MB 33.5MB/s 
     |████████████████████████████████| 61kB 22.0MB/s 
  Created wheel for m2r: filename=m2r-0.2.1-cp36-none-any.whl size=10538 sha256=1dcb269ef2177ee7043f11014f9b7310b1f202f0c6491f883fd020b70588a7f5
  Stored in directory: /root/.cache/pip/wheels/47/f8/dc/80f56bc4abf785834d422c2f5c864a14bf34576612aeb03492
Successfully built m2r
ERROR: magenta 0.3.19 has requirement tensorflow>=1.12.0, but you'll have tensorflow 1.11.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=0.2

In [2]:
#import other necessary libraries to help with the analysis
import logging
import os
import pickle

from keras import Model
from keras.layers import Dense
from sklearn.model_selection import train_test_split

from keras_pandas import lib
from keras_pandas.Automater import Automater

Using TensorFlow backend.


In [3]:
save_results = True
# TODO Load data
observations = lib.load_titanic()
print('Observation columns: {}'.format(list(observations.columns)))

Observation columns: ['survived', 'pclass', 'name', 'sex', 'age', 'siblings_spouses_aboard', 'parents_children_aboard', 'fare']


In [0]:
# TODO Train /test split
train_observations, test_observations = train_test_split(observations)
train_observations = train_observations.copy()
test_observations = test_observations.copy()

In [0]:
# TODO List out variable types

data_type_dict = {'numerical': ['age', 'siblings_spouses_aboard', 'parents_children_aboard', 'fare'],
                      'categorical': ['survived', 'pclass', 'sex'],
                      'text': ['name'],
                        'timeseries': []}
output_var = 'survived'

In [6]:
# Create and fit Automater
auto = Automater(data_type_dict=data_type_dict, output_var=output_var)
auto.fit(train_observations)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/keras_pandas/transformations.py:357: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/keras_pandas/transformations.py:393: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/keras_pandas/data_types/Numerical.py:36: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  transfor

In [7]:
# Transform data
train_X, train_y = auto.fit_transform(train_observations)
test_X, test_y = auto.transform(test_observations)

/usr/local/lib/python3.6/dist-packages/keras_pandas/transformations.py:357: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/keras_pandas/transformations.py:393: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/keras_pandas/data_types/Numerical.py:36: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  transformed = transformed_observations[variable].as_matrix()
/usr/local/lib/python3.6/dist-packages/keras_pandas/data_types/Categorical.py:41: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  transformed = transformed_observations[variable].as_matrix()


In [0]:
# TODO Create and fit keras (deep learning) model.

x = auto.input_nub
x = Dense(32)(x)
x = Dense(32)(x)
x = auto.output_nub(x)

In [0]:
model = Model(inputs=auto.input_layers, outputs=x)
model.compile(optimizer='adam', loss=auto.suggest_loss())

In [10]:
model.fit(train_X, train_y)

Epoch 1/1
665/665 [==============================] - 2s 3ms/step - loss: 0.8595


In [11]:
# Make model predictions and inverse transform model predictions, to get usable results
pred_test_y = model.predict(test_X)
auto.inverse_transform_output(pred_test_y)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0])